## Pause Capacity

Fabric Capacity is an Azure Resource.  In order to be able to manipulate the capacity we need to leverage the azure management APIs.  If you need to keep, but stop incuring charges for provisioned Fabric capacity you use the suspend method of the resource.

I was unable to find API documentation for this API.  I inferred the capabilities existed by referencing the Power Shell libraries for Power BI Embedded capacities:  https://learn.microsoft.com/en-us/powershell/module/az.powerbiembedded/suspend-azpowerbiembeddedcapacity?view=azps-11.5.0

In [ ]:
pip install requests msal

In [ ]:
#This leverages the code encapsulated in services/aadservice.py that encapsulates the service principle login.
#Note: for most of the other notebooks we're not passing in a scope, this is because the code in aadservice.property
#defaults the scope to a scope for the Power BI API.   Since we're calling the azure management API we need a different
#scope.
from services.aadservice import AadService
scope = 'https://management.azure.com/.default'
aad_token = AadService.get_access_token(scope)

headers =  {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + aad_token}

In [ ]:
#Note the Service Principal calling the rest API needs to be granted contributor access to whatever resource group you 
#use here.  Otherwise the service principal
resource_group = "cgmpbie"  

capacity_name = "cgmthisisatest"

#You need to set your own subscription id.
subscription_id = "7258b7d4-3429-4998-815b-8cd6954b7ef9"

#This can be an Email, or the object id (not application id) of the Service Prinicipal.
administrators = ['chmitch@microsoftanalytics.info','6709b293-4789-477f-aee2-607b7139e63c']

In [ ]:
import requests
import json

api_url = f'https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.Fabric/capacities/{capacity_name}/suspend?api-version=2022-07-01-preview'       

api_response = requests.post(api_url, headers=headers)#, data=json.dumps(body))

#error handling for create capacity
if api_response.status_code != 202:
    description = f'Error pausing capacity:\n  -Status Code:\t{api_response.status_code}\n  -Reason:\t{api_response.reason}\n  -RequestId:\t{api_response.headers.get("RequestId")}\n  -Text:\t{api_response.text}'
    print(description)
else:
    print("Capacity paused")